# Project 3

**Group:** Andy Guan (265559) & Parsa Moniri (272030)

## Topic 2, Task 1

In [ ]:
import importlib, main
importlib.reload(main)

from main import random_walk, random_walk_fast
_ = random_walk(iterations=2000, walker_count=6, step_size=1.0, L=50.0, seed=1)
_ = random_walk_fast(steps_per_walker=2000, walker_count=6, step_size=1.0, L=50.0, seed=1)
